# Lab 5_1 Auto Encoder

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [3]:

# Load MNIST Dataset
transform = transforms.ToTensor()
dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [4]:
# Define Autoencoder with Classification
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder Layers
        self.fc1 = nn.Linear(784, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 16)

        # Decoder Layers
        self.fc4 = nn.Linear(16, 64)
        self.relu3 = nn.ReLU()
        self.fc5 = nn.Linear(64, 256)
        self.relu4 = nn.ReLU()
        self.fc6 = nn.Linear(256, 784)
        self.sigmoid = nn.Sigmoid()

        # Classification Layers
        self.fc7 = nn.Linear(16, 10)

    def encode(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

    def decode(self, x):
        x = self.fc4(x)
        x = self.relu3(x)
        x = self.fc5(x)
        x = self.relu4(x)
        x = self.fc6(x)
        x = self.sigmoid(x)
        return x

    def classify(self, x):
        x = self.fc7(x)
        return x

    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        y_pred = self.classify(encoded)
        return decoded, y_pred

In [5]:

# Initialize Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder().to(device)
criterion_recon = nn.MSELoss()
criterion_class = nn.CrossEntropyLoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [6]:
# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.view(images.size(0), -1).to(device)  # Flatten images
        labels = labels.to(device)

        outputs, y_pred = autoencoder(images)  # Forward pass
        loss_recon = criterion_recon(outputs, images)  # Reconstruction loss
        loss_class = criterion_class(y_pred, labels)  # Classification loss
        loss = loss_recon + loss_class  # Total loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training complete.")

Epoch [1/10], Loss: 0.2106
Epoch [2/10], Loss: 0.2104
Epoch [3/10], Loss: 0.1123
Epoch [4/10], Loss: 0.0941
Epoch [5/10], Loss: 0.3006
Epoch [6/10], Loss: 0.0752
Epoch [7/10], Loss: 0.0291
Epoch [8/10], Loss: 0.0746
Epoch [9/10], Loss: 0.0220
Epoch [10/10], Loss: 0.0493
Training complete.


In [7]:
# Testing Loop - Compute Accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(images.size(0), -1).to(device)
        labels = labels.to(device)

        encoded = autoencoder.encode(images)
        y_pred_test = autoencoder.classify(encoded)
        predicted = torch.argmax(y_pred_test, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 97.49%
